In [1]:
!pip install sentence-transformers

In [2]:
!pip install huggingface_hub

In [3]:
!pip install langchain

In [4]:
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [5]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 24.0 from /Users/Siar/Desktop/AWT-repos/generative-agents/venv/lib/python3.9/site-packages/pip (python 3.9)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.5 MB/s eta 0:00:00 0:00:01
  Running command pip subprocess to install build dependencies
    Using cached setuptools-69.0.3-py3-none-any.whl.metadata (6.3 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl.metadata (14 kB)
    Using cached ninja-1.11.1.1-py2.py3-none-macosx_10_9_universal2.macosx_10_9_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl.metadata (5.3 kB)
    Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
    Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached setuptools-69.0.3-py3-none-any.whl (819 kB)
  Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/48.5 MB ? eta -:--:--

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from huggingface_hub import hf_hub_download
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import CTransformers
from langchain.llms import LlamaCpp
import numpy as np
import sys
import torch
import faiss
import requests
import random
from llama_cpp import Llama

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

In [ ]:
torch.cuda.empty_cache()

# Change the model name and store it in a variable
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in bin format

# Change the FAISS index path to use the local file
DB_FAISS_PATH = "/vectorstore/db_faiss/faiss.index"

# Download the model using HF-Hub
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)

# Load the dataset into a pandas DataFrame
loader = CSVLoader(file_path="/data/mc-dataset.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

# Change the text splitter import to use the langchain version
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=20
)

# Split the dataset into text chunks
text_chunks = text_splitter.split_documents(data)

text_strings = [doc.page_content for doc in text_chunks]

# Initialize the model from the sentence-transformers library
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

def get_embeddings(texts):
    # texts should be a list of strings
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings

# Change the embedding import to use the langchain version
embeddings = get_embeddings(text_strings)

# Extract the embeddings from the FAISS index
#docsearch = FAISS.from_documents(text_chunks, embeddings)

#docsearch.save_local(DB_FAISS_PATH)
dimension = embeddings.shape[1]  # Get the dimensionality of your embeddings
index = faiss.IndexFlatL2(dimension)  # Create a flat (brute-force) search index

# FAISS requires the data type to be float32
if embeddings.dtype != np.float32:
    embeddings = embeddings.astype(np.float32)

index.add(embeddings)  # Add your embeddings to the index

# To save the index to disk
faiss.write_index(index, DB_FAISS_PATH)

#NEUNEUNEU
def fetch_question_from_api(question_id):
    """Fetch a question from the API."""
    api_url = f'http://3.139.84.244:8000/questions/{question_id}'
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching question: HTTP {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Exception when fetching question: {e}")
        return None


def generate_exam(num_questions=50):
    """Generate an exam with a specified number of questions."""
    exam = []
    question_ids = random.sample(range(1, 490), num_questions)  # Randomly select question IDs

    for q_id in question_ids:
        question = fetch_question_from_api(q_id)
        if question:
            exam.append(question)
        else:
            print(f"Failed to fetch question with ID: {q_id}")

    return exam

def generate_response(agent, prompt):
    """Generate a response using the specified agent."""
    response = agent(
        prompt=prompt,
        max_tokens=1000,
        temperature=0.5,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['USER:', 'ASSISTANT:'],
        echo=True
    )
    return response["choices"][0]["text"]


def agent_interaction(agent_1, agent_2, question_id):
    """Simulate the interaction between two agents."""
    question_data = fetch_question_from_api(question_id)
    if not question_data:
        print("Question not found.")
        return

    question = question_data['Question']
    choices = f"{question_data['Choice_A']} {question_data['Choice_B']} {question_data['Choice_C']} {question_data['Choice_D']} {question_data['Choice_E']}"
    correct_answer = question_data['Correct_Answer']

    # Agent 1 asks the question
    prompt_for_agent_2 = f"Question: {question} {choices}\nAnswer:"
    response_from_agent_2 = generate_response(agent_2, prompt_for_agent_2)

    # Print the agent's response (only the letter)
    print(f"Agent 2's Response: {response_from_agent_2}")

    # Agent 1 provides the full correct answer from the API
    full_correct_answer = f"The correct answer is: {correct_answer}, which is {question_data[f'Choice_{correct_answer}']}"
    print(full_correct_answer)


def main():
    for question_id in range(1, 11):  # Loop through questions 1 to 10
        agent_interaction(lcpp_llm, lcpp_llm, question_id)
        print("-" * 30)


if __name__ == "__main__":
    main()